In [24]:
import glob
import os
from pathlib import Path
from collections import defaultdict

import numpy as np
import pandas as pd
from scipy.ndimage import gaussian_filter1d

import matplotlib.pyplot as plt
import matplotlib as mpl
from matplotlib.ticker import (MultipleLocator, AutoMinorLocator)

In [25]:
def find_csv_files(log_dir, algos, filename) -> list[Path]:
    # Matches …/<SEED>/debug/**/finetuning_eval.csv  (depth under debug doesn't matter)
    
    csv_files = dict()
    for algo, env_name, exp_log_dir in algos:
        pattern = os.path.join(log_dir, algo, exp_log_dir, "*", "debug", "**", filename)
        files = [p for p in glob.glob(pattern, recursive=True)]
        if env_name not in csv_files:
            csv_files[env_name] = {algo: files}
        else:
            csv_files[env_name][algo] = files 
    return csv_files


log_dir = "/n/fs/rl-chongyiz/exp_logs/ogbench_logs"
algos = [
    ("sarsa_ifql_vib_gpi_offline2offline", "cheetah_run", "20250510_sarsa_ifql_vib_gpi_offline2offline_cheetah_run_obs_norm=normal_alpha=0.3_ft_size=500000_ft_steps=250000_eval_freq=1000_num_fg=16_actor_freq=4_expectile=0.9_actor_ln=False_kl_weight=0.05_latent_dim=128"),
    # ("sarsa_ifql_vib_gpi_offline2offline", "cheetah_run", "20250517_sarsa_ifql_vib_gpi_offline2offline_cheetah_run_obs_norm=normal_alpha=0.3_ft_size=500000_ft_steps=250000_eval_freq=1000_num_fg=16_actor_freq=4_expectile=0.9_actor_ln=False_kl_weight=0.05_latent_dim=128"),
    ("rebrac_offline2offline", "cheetah_run", "20250510_rebrac_offline2offline_cheetah_run_obs_norm_type=normal_alpha_actor=0.1_alpha_critic=0.1_ft_size=500000_ft_steps=250000_eval_freq=1000_actor_freq=4"),
    ("crl_infonce_offline2offline", "cheetah_run", "20250510_crl_infonce_offline2offline_cheetah_run_obs_norm_type=normal_alpha=0.03_ft_size=500000_ft_steps=250000_eval_freq=1000_reward_type=state_actor_freq=4"),
    ("dino_rebrac_offline2offline", "cheetah_run", "20250511_dino_rebrac_offline2offline_cheetah_run_obs_norm_type=normal_alpha_actor=0.1_alpha_critic=0.1_actor_freq=4_repr_noise=0.2_repr_noise_clip=0.2_repr_temp=0.1_target_repr_temp=0.04_short"),
    ("mbpo_rebrac_offline2offline", "cheetah_run", "20250515_mbpo_rebrac_offline2offline_cheetah_run_obs_norm_type=normal_ft_size=500000_ft_steps=250000_eval_freq=1000_alpha_actor=0.1_alpha_critic=0.1_num_model_rollouts=256_num_model_rollout_steps=2_actor_freq=4"),
    ("fb_repr_offline2offline", "cheetah_run", "20250512_fb_repr_offline2offline_cheetah_run_obs_norm_type=normal_ft_size=500000_ft_steps=250000_eval_freq=1000_repr_alpha=1.0_awr_alpha=1.0_expectile=0.9_actor_freq=4"),
    
    ("sarsa_ifql_vib_gpi_offline2offline", "cheetah_run_backward", "20250510_sarsa_ifql_vib_gpi_offline2offline_cheetah_run_backward_obs_norm=normal_alpha=0.3_ft_size=500000_ft_steps=250000_eval_freq=1000_num_fg=16_actor_freq=4_expectile=0.9_actor_ln=False_kl_weight=0.05_latent_dim=128"),
    ("rebrac_offline2offline", "cheetah_run_backward", "20250510_rebrac_offline2offline_cheetah_run_backward_obs_norm_type=normal_alpha_actor=0.1_alpha_critic=0.1_ft_size=500000_ft_steps=250000_eval_freq=1000_actor_freq=4"),
    ("crl_infonce_offline2offline", "cheetah_run_backward", "20250510_crl_infonce_offline2offline_cheetah_run_backward_obs_norm_type=normal_alpha=0.003_ft_size=500000_ft_steps=250000_eval_freq=1000_reward_type=state_actor_freq=4"),
    
    # ("sarsa_ifql_vib_gpi_offline2offline", "quadruped_jump", "20250518_sarsa_ifql_vib_gpi_offline2offline_quadruped_jump_obs_norm=normal_alpha=0.3_ft_size=500000_ft_steps=250000_eval_freq=1000_num_fg=16_actor_freq=4_expectile=0.9_value_ln=True_kl_weight=0.005_latent_dim=512"),
    ("sarsa_ifql_vib_gpi_offline2offline", "quadruped_jump", "20250518_sarsa_ifql_vib_gpi_offline2offline_quadruped_jump_obs_norm=normal_alpha=0.3_ft_size=500000_ft_steps=250000_eval_freq=1000_num_fg=16_actor_freq=4_expectile=0.9_value_ln=True_kl_weight=0.005_latent_dim=512"),
    # ("sarsa_ifql_offline2offline", "quadruped_jump", "20250406_sarsa_ifql_offline2offline_quadruped_jump_obs_norm=normal_alpha=3.0_ft_size=500000_ft_steps=250000_eval_freq=2000_num_fg=16_actor_freq=4_expectile=0.75_q_agg=min_critic_fm_loss=sarsa_squared_reward=state_action_bc_pretrain"),
    # ("iql_offline2offline", "quadruped_jump", "20250407_iql_offline2offline_quadruped_jump_obs_norm_type=normal_alpha=10.0_ft_size=500000_ft_steps=250000_eval_freq=2000_expectile=0.9_actor_freq=4"),
    ("rebrac_offline2offline", "quadruped_jump", "20250406_rebrac_offline2offline_quadruped_jump_obs_norm_type=normal_alpha_actor=1.0_alpha_critic=1.0_ft_size=500000_ft_steps=250000_eval_freq=2000_actor_freq=4"),
    ("crl_infonce_offline2offline", "quadruped_jump", "20250511_crl_infonce_offline2offline_quadruped_jump_obs_norm_type=normal_alpha=0.03_ft_size=500000_ft_steps=250000_eval_freq=1000_reward_type=state_actor_freq=4"),
    ("mbpo_rebrac_offline2offline", "quadruped_jump", "20250515_mbpo_rebrac_offline2offline_quadruped_jump_obs_norm_type=normal_ft_size=500000_ft_steps=250000_eval_freq=1000_alpha_actor=0.1_alpha_critic=0.1_num_model_rollouts=256_num_model_rollout_steps=2_actor_freq=4"),
    ("dino_rebrac_offline2offline", "quadruped_jump", "20250511_dino_rebrac_offline2offline_quadruped_jump_obs_norm_type=normal_alpha_actor=0.1_alpha_critic=0.1_actor_freq=4_repr_noise=0.2_repr_noise_clip=0.2_repr_temp=0.1_target_repr_temp=0.04_short"),
    ("fb_repr_offline2offline", "quadruped_jump", "20250512_fb_repr_offline2offline_quadruped_jump_obs_norm_type=normal_ft_size=500000_ft_steps=250000_eval_freq=1000_repr_alpha=10.0_awr_alpha=1.0_expectile=0.9_actor_freq=4"),

    # ("sarsa_ifql_vib_gpi_offline2offline", "quadruped_run", "20250511_sarsa_ifql_vib_gpi_offline2offline_quadruped_run_obs_norm=normal_alpha=0.3_ft_size=500000_ft_steps=250000_eval_freq=1000_num_fg=16_actor_freq=4_expectile=0.9_actor_ln=False_kl_weight=0.005_latent_dim=128"),
    ("sarsa_ifql_vib_gpi_offline2offline", "quadruped_run", "20250517_sarsa_ifql_vib_gpi_offline2offline_quadruped_run_obs_norm=normal_alpha=0.3_ft_size=500000_ft_steps=250000_eval_freq=1000_num_fg=16_actor_freq=1_expectile=0.9_actor_ln=False_kl_weight=0.005_latent_dim=128"),
    ("rebrac_offline2offline", "quadruped_run", "20250511_rebrac_offline2offline_quadruped_run_obs_norm_type=normal_alpha_actor=1.0_alpha_critic=1.0_ft_size=500000_ft_steps=250000_eval_freq=1000_actor_freq=4"),
    ("crl_infonce_offline2offline", "quadruped_run", "20250511_crl_infonce_offline2offline_quadruped_run_obs_norm_type=normal_alpha=0.03_ft_size=500000_ft_steps=250000_eval_freq=1000_reward_type=state_actor_freq=4"),
    ("dino_rebrac_offline2offline", "quadruped_run", "20250511_dino_rebrac_offline2offline_quadruped_run_obs_norm_type=normal_alpha_actor=0.1_alpha_critic=0.1_actor_freq=4_repr_noise=0.2_repr_noise_clip=0.2_repr_temp=0.1_target_repr_temp=0.04_short"),
    ("fb_repr_offline2offline", "quadruped_run", "20250512_fb_repr_offline2offline_quadruped_run_obs_norm_type=normal_ft_size=500000_ft_steps=250000_eval_freq=1000_repr_alpha=10.0_awr_alpha=1.0_expectile=0.9_actor_freq=4"),

    ("sarsa_ifql_offline2offline", "cube-single-play-singletask-task2-v0", "20250407_sarsa_ifql_offline2offline_cube-single-play-singletask-task2-v0_obs_norm=normal_alpha=30.0_ft_size=500000_ft_steps=250000_eval_freq=2000_num_fg=16_actor_freq=4_expectile=0.8_q_agg=mean_critic_fm_loss=sarsa_squared_reward=state_action_bc_pretrain"),
    ("iql_offline2offline", "cube-single-play-singletask-task2-v0", "20250407_iql_offline2offline_cube-single-play-singletask-task2-v0_obs_norm_type=normal_alpha=1.0_ft_size=500000_ft_steps=250000_eval_freq=2000_expectile=0.9_actor_freq=4"),
    ("rebrac_offline2offline", "cube-single-play-singletask-task2-v0", "20250407_rebrac_offline2offline_cube-single-play-singletask-task2-v0_obs_norm_type=normal_alpha_actor=1.0_alpha_critic=1.0_ft_size=500000_ft_steps=250000_eval_freq=2000_actor_freq=4")
]


env_name_maps = {
    'cheetah_run': 'cheetah run',
    'cheetah_run_backward': 'cheetah run backward',
    'quadruped_run': 'quadruped run',
    'quadruped_jump': 'quadruped jump',
    'walker_walk': 'walker walk',
    'cube-single-play-singletask-task2-v0': 'cube single front',
}
algo_name_maps = {
    'sarsa_ifql_vib_gpi_offline2offline': 'InFOM (Ours)',
    'sarsa_ifql_offline2offline': 'InFOM (Ours)',
    'iql_offline2offline': 'IQL',
    'rebrac_offline2offline': 'ReBRAC',
    'crl_infonce_offline2offline': 'CRL + IS',
    'mbpo_rebrac_offline2offline': 'MBPO + ReBRAC',
    'dino_rebrac_offline2offline': 'DINO + ReBRAC',
    'fb_repr_offline2offline': 'FB + IQL',
}


csv_files = find_csv_files(log_dir, algos, filename="finetuning_eval.csv")

In [26]:
def load_data(csv_path, stat_name, step_name) -> np.ndarray:
    df = pd.read_csv(csv_path)
    if stat_name not in df.columns:
        raise KeyError(f"{csv_path} doesn't contain {stat_name}")
    x = df[step_name].values
    y = df[stat_name].values
    return dict(x=x, y=y)


algo_data = defaultdict(dict)
for env_name, env_csv_files in csv_files.items():
    for algo, csv_files in env_csv_files.items():
        seed_data = []
        for csv_file in csv_files:
            if "singletask" in env_name:
                data = load_data(csv_file, "evaluation/episode.success", "step")
            else:
                data = load_data(csv_file, "evaluation/episode.return", "step")
            if len(seed_data) == 0:
                seed_data.append(data["x"])
            else:
                assert np.all(data["x"] == seed_data[0])
            seed_data.append(data["y"])

        seed_data = np.asarray(seed_data)
        # steps = seed_data[0]
        # data = seed_data[1:]
        
        algo_data[env_name_maps[env_name]][algo_name_maps[algo]] = seed_data
        if len(seed_data) == 1:
            print("Warning: only one random seed!")

In [27]:
print("cheetah run")
print(algo_data['cheetah run']['InFOM (Ours)'].shape)
print(algo_data['cheetah run']['ReBRAC'].shape)
print(algo_data['cheetah run']['CRL + IS'].shape)
print(algo_data['cheetah run']['DINO + ReBRAC'].shape)
print(algo_data['cheetah run']['MBPO + ReBRAC'].shape)
print(algo_data['cheetah run']['FB + IQL'].shape)

print("quadruped jump")
print(algo_data['quadruped jump']['InFOM (Ours)'].shape)
print(algo_data['quadruped jump']['ReBRAC'].shape)
print(algo_data['quadruped jump']['CRL + IS'].shape)
print(algo_data['quadruped jump']['DINO + ReBRAC'].shape)
print(algo_data['quadruped jump']['MBPO + ReBRAC'].shape)
print(algo_data['quadruped jump']['FB + IQL'].shape)

print("quadruped run")
print(algo_data['quadruped run']['InFOM (Ours)'].shape)
print(algo_data['quadruped run']['ReBRAC'].shape)
print(algo_data['quadruped run']['CRL + IS'].shape)
print(algo_data['quadruped run']['DINO + ReBRAC'].shape)
print(algo_data['quadruped run']['FB + IQL'].shape)

print("cube single front")
print(algo_data['cube single front']['InFOM (Ours)'].shape)
print(algo_data['cube single front']['IQL'].shape)
print(algo_data['cube single front']['ReBRAC'].shape)

In [28]:
# fig, axes = plt.subplots(1, 3, figsize=(12, 3.2), gridspec_kw=dict(wspace=0.16, hspace=0.5))
fig, axes = plt.subplots(1, 5, figsize=(20, 3.2), gridspec_kw=dict(wspace=0.22, hspace=0.5))

for (env_name, env_data), ax in zip(algo_data.items(), axes):
    for algo, data in env_data.items():
        # if ax in [axes[0], axes[1]]:
        #     x = data[0][::2] - 1e6
        #     y = data[1:, ::2]
        # else:
        #     x = data[0] - 1e6
        #     y = data[1:]
        x = data[0] - 1e6
        y = data[1:]
        y = gaussian_filter1d(y, 0.5)
        y_mean = np.mean(y, axis=0)
        y_std = np.std(y, axis=0, ddof=1)
        
        l, = ax.plot(x, y_mean, label=algo, zorder=3)
        ax.fill_between(x, y_mean - y_std, y_mean + y_std, alpha=0.3, lw=0, color=l.get_color(), zorder=3)

    if ax == axes[0]:
        ax.set_ylabel('return', fontsize=14)
        ax.set_xlim([-4e2, 5e4 + 4e2])
        ax.set_ylim([-5, 120 + 5])
        ax.set_yticks([0, 120])
        ax.yaxis.set_minor_locator(MultipleLocator(100))
        
        # ax.legend(bbox_to_anchor=(0.3, -0.2),
        #     loc="upper left", labelspacing=1, columnspacing=1.8075, fancybox=False,
        #     shadow=False, fontsize=12.5, borderpad=0.5, handlelength=1.7, ncol=3)
    if ax == axes[1]:
        ax.set_ylabel('return', fontsize=14)
        ax.set_xlim([-4e2, 2.5e5 + 4e2])
        ax.set_ylim([-5, 120 + 5])
        ax.set_yticks([0, 120])
        ax.yaxis.set_minor_locator(MultipleLocator(100))
    if ax == axes[2]:
        ax.set_ylabel('return', fontsize=14)
        ax.set_xlim([-4e2, 5e4 + 4e2])
        ax.set_ylim([-25, 825])
        ax.set_yticks([0, 400, 800])
        ax.yaxis.set_minor_locator(MultipleLocator(100))
    elif ax == axes[3]:
        ax.set_ylabel('return', fontsize=14)
        ax.set_xlim([-4e2, 10e4 + 4e2])
        ax.set_ylim([-25, 825])
        ax.set_yticks([0, 400, 800])
        ax.yaxis.set_minor_locator(MultipleLocator(100))
    elif ax == axes[4]:
        ax.set_ylabel('success rate', fontsize=14)
        ax.set_xlim([-1.25e4, 0.25e6 + 1.25e4])
        ax.set_ylim([-0.03, 1.0 + 0.03])
        ax.set_yticks([0.0, 0.5, 1.0])
        ax.yaxis.set_minor_locator(MultipleLocator(0.1))
    
    ax.legend()
    
    ax.set_title(env_name, fontsize=14)
    ax.set_xlabel('fine-tuning steps', fontsize=12)
    ax.ticklabel_format(axis='x', style='sci', scilimits=(0, 0))  
    
    ax.grid(zorder=3)     

fig.tight_layout()
filepath = "/u/cz8792/research/ogbench/plot_scripts/figures/convergence_speed_ablation_lc.pdf"
fig.savefig(filepath, dpi=150, bbox_inches="tight")
plt.show()


In [34]:
# fig, axes = plt.subplots(1, 2, figsize=(8, 3.2), gridspec_kw=dict(wspace=0.15, hspace=0.5))
fig, axes = plt.subplots(1, 2, figsize=(6.4, 3.4))

for env_name, env_data in algo_data.items():
    if env_name not in ['cheetah run', 'quadruped jump']:
        continue
    
    if env_name == 'cheetah run':
        ax = axes[0]
    else:
        ax = axes[1]
    
    for algo, data in env_data.items():
        x = data[0] - 1e6
        y = data[1:]
        y = gaussian_filter1d(y, 0.75)
        y_mean = np.mean(y, axis=0)
        y_std = np.std(y, axis=0, ddof=1)
        
        l, = ax.plot(x, y_mean, label=algo, zorder=3)
        if algo == 'InFOM (Ours)':
            ax.fill_between(x, y_mean - y_std, y_mean + y_std, alpha=0.3, lw=0, color=l.get_color(), zorder=5)
        else:
            ax.fill_between(x, y_mean - y_std, y_mean + y_std, alpha=0.3, lw=0, color=l.get_color(), zorder=3)

    if ax == axes[0]:
        ax.set_ylabel('return', fontsize=14)
        ax.set_xlim([-1e3, 5e4 + 1e3])
        ax.set_ylim([-3, 120 + 3])
        ax.set_yticks([0, 60, 120])
        ax.yaxis.set_minor_locator(MultipleLocator(20))
        
        ax.legend(bbox_to_anchor=(0.1, -0.22),
            loc="upper left", labelspacing=1, columnspacing=1.2075, fancybox=False,
            shadow=False, fontsize=11, borderpad=0.35, handlelength=1.7, ncol=3)
    if ax == axes[1]:
        # ax.set_ylabel('return', fontsize=14)
        ax.set_xlim([-1e3, 5e4 + 1e3])
        ax.set_ylim([-20, 820])
        ax.set_yticks([0, 400, 800])
        ax.yaxis.set_minor_locator(MultipleLocator(100))
    
    ax.set_title(env_name, fontsize=14)
    ax.set_xlabel('fine-tuning steps', fontsize=12)
    ax.ticklabel_format(axis='x', style='sci', scilimits=(0, 0))  
    
    ax.grid(zorder=3)     

# fig.tight_layout()
fig.tight_layout(rect=(-0.026, -0.06, 1.02, 1.04))  # rect = (left, bottom, right, top), default: (0, 0, 1, 1)
fig.subplots_adjust(wspace=0.18)
filepath = "/u/cz8792/research/ogbench/plot_scripts/figures/convergence_speed_ablation_lc.pdf"
# fig.savefig(filepath, dpi=300, bbox_inches="tight")
fig.savefig(filepath, dpi=300)
plt.show()
